In [1]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"GPU name: {torch.cuda.get_device_name(0)}")

# 테스트
x = torch.tensor([1.0, 2.0, 3.0]).cuda()
print(f"GPU tensor: {x}")

PyTorch: 2.7.1+cu118
CUDA available: True
CUDA version: 11.8
GPU name: NVIDIA GeForce GTX 1060 3GB
GPU tensor: tensor([1., 2., 3.], device='cuda:0')


In [2]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

# 이제 작동해야 함
if torch.cuda.is_available():
    gpu_tensor = torch.cuda.FloatTensor([1, 2, 3])
    print(f"GPU tensor: {gpu_tensor}")

PyTorch: 2.7.1+cu118
CUDA available: True
CUDA version: 11.8
GPU tensor: tensor([1., 2., 3.], device='cuda:0')


C:\Users\602-01\AppData\Local\Temp\ipykernel_8580\770461706.py:8: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\tensor\python_tensor.cpp:80.)
  gpu_tensor = torch.cuda.FloatTensor([1, 2, 3])


In [3]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
cpu = torch.FloatTensor([1, 2, 3])
gpu = torch.cuda.FloatTensor([1, 2, 3])
tensor = torch.rand((1, 1), device=device)
print(device)
print(cpu)
print(gpu)
print(tensor)

cuda
tensor([1., 2., 3.])
tensor([1., 2., 3.], device='cuda:0')
tensor([[0.0558]], device='cuda:0')


In [5]:
print(torch.cuda.is_available())

True


In [ ]:
# GPU 메모리 사용량 모니터링
print(f"GPU Memory: {torch.cuda.memory_allocated()/1024**2:.1f}MB / {torch.cuda.max_memory_allocated()/1024**2:.1f}MB")

GPU Memory: 0.0MB / 0.0MB


In [4]:
import torch
import time
import gc

def limit_performance_test():
    print("GTX 1060 3GB 한계 성능 테스트")
    print("=" * 60)
    
    device = torch.device('cuda')
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"총 GPU 메모리: {total_memory:.2f} GB")
    
    def cleanup():
        torch.cuda.empty_cache()
        gc.collect()
        torch.cuda.synchronize()
    
    def get_memory_usage():
        return torch.cuda.memory_allocated() / 1024**2
    
    # 1. 최대 단일 텐서 크기 한계 테스트
    print("\n1. 최대 단일 텐서 크기 한계")
    cleanup()
    
    max_size = 0
    for size in range(1000, 20000, 200):
        cleanup()
        try:
            tensor = torch.randn(size, size, device=device, dtype=torch.float32)
            memory_used = get_memory_usage()
            print(f"크기 {size}x{size}: {memory_used:.1f}MB 성공")
            max_size = size
            del tensor
            
            if memory_used > 2800:  # 거의 한계에 근접
                break
                
        except RuntimeError as e:
            print(f"크기 {size}x{size}: 메모리 한계 도달")
            break
    
    print(f"최대 단일 텐서: {max_size}x{max_size}")
    
    # 2. 최대 배치 크기 테스트 (ResNet-like 모델)
    print("\n2. 최대 배치 크기 테스트 (224x224 이미지)")
    cleanup()
    
    # 실제 ResNet 구조와 유사한 모델
    class TestCNN(torch.nn.Module):
        def __init__(self):
            super().__init__()
            self.features = torch.nn.Sequential(
                torch.nn.Conv2d(3, 64, 7, stride=2, padding=3),
                torch.nn.BatchNorm2d(64),
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(3, stride=2, padding=1),
                
                torch.nn.Conv2d(64, 128, 3, padding=1),
                torch.nn.BatchNorm2d(128),
                torch.nn.ReLU(),
                torch.nn.Conv2d(128, 128, 3, padding=1),
                torch.nn.BatchNorm2d(128),
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(2),
                
                torch.nn.Conv2d(128, 256, 3, padding=1),
                torch.nn.BatchNorm2d(256),
                torch.nn.ReLU(),
                torch.nn.Conv2d(256, 256, 3, padding=1),
                torch.nn.BatchNorm2d(256),
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(2),
                
                torch.nn.AdaptiveAvgPool2d(1)
            )
            self.classifier = torch.nn.Linear(256, 1000)
        
        def forward(self, x):
            x = self.features(x)
            x = x.view(x.size(0), -1)
            x = self.classifier(x)
            return x
    
    model = TestCNN().to(device)
    max_batch = 0
    
    for batch_size in range(1, 65):
        cleanup()
        try:
            input_data = torch.randn(batch_size, 3, 224, 224, device=device)
            with torch.no_grad():
                output = model(input_data)
            
            memory_used = get_memory_usage()
            print(f"배치 크기 {batch_size}: {memory_used:.1f}MB")
            max_batch = batch_size
            
            del input_data, output
            
            if memory_used > 2800:
                break
                
        except RuntimeError:
            print(f"배치 크기 {batch_size}: 메모리 한계")
            break
    
    print(f"최대 배치 크기 (224x224): {max_batch}")
    
    # 3. 훈련 시 최대 배치 크기 (그래디언트 포함)
    print("\n3. 훈련 시 최대 배치 크기")
    cleanup()
    
    model.train()
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
    
    max_train_batch = 0
    for batch_size in range(1, max_batch + 1):
        cleanup()
        try:
            input_data = torch.randn(batch_size, 3, 224, 224, device=device)
            target = torch.randint(0, 1000, (batch_size,), device=device)
            
            optimizer.zero_grad()
            output = model(input_data)
            loss = criterion(output, target)
            loss.backward()
            
            memory_used = get_memory_usage()
            print(f"훈련 배치 {batch_size}: {memory_used:.1f}MB")
            max_train_batch = batch_size
            
            del input_data, target, output, loss
            
            if memory_used > 2700:  # 훈련은 더 보수적으로
                break
                
        except RuntimeError:
            print(f"훈련 배치 {batch_size}: 메모리 한계")
            break
    
    print(f"최대 훈련 배치 크기: {max_train_batch}")
    
    # 4. 다양한 이미지 해상도별 최대 배치
    print("\n4. 해상도별 최대 배치 크기")
    cleanup()
    
    resolutions = [64, 128, 224, 320, 512]
    model_simple = torch.nn.Sequential(
        torch.nn.Conv2d(3, 64, 3, padding=1),
        torch.nn.ReLU(),
        torch.nn.Conv2d(64, 128, 3, padding=1),
        torch.nn.ReLU(),
        torch.nn.AdaptiveAvgPool2d(1),
        torch.nn.Flatten(),
        torch.nn.Linear(128, 10)
    ).to(device)
    
    for res in resolutions:
        cleanup()
        max_batch_res = 0
        
        for batch_size in range(1, 100):
            try:
                input_data = torch.randn(batch_size, 3, res, res, device=device)
                with torch.no_grad():
                    output = model_simple(input_data)
                
                memory_used = get_memory_usage()
                max_batch_res = batch_size
                
                del input_data, output
                
                if memory_used > 2800:
                    break
                    
            except RuntimeError:
                break
        
        print(f"해상도 {res}x{res}: 최대 배치 {max_batch_res}")
    
    # 5. 행렬 곱셈 절대 한계
    print("\n5. 행렬 곱셈 절대 한계")
    cleanup()
    
    max_matmul_size = 0
    for size in range(3000, 10000, 100):
        cleanup()
        try:
            a = torch.randn(size, size, device=device)
            b = torch.randn(size, size, device=device)
            
            start_time = time.time()
            c = torch.mm(a, b)
            torch.cuda.synchronize()
            compute_time = time.time() - start_time
            
            memory_used = get_memory_usage()
            gflops = (2 * size**3) / (compute_time * 1e9)
            
            print(f"크기 {size}: {compute_time:.3f}초, {gflops:.0f} GFLOPS, {memory_used:.1f}MB")
            max_matmul_size = size
            
            del a, b, c
            
            if memory_used > 2900:
                break
                
        except RuntimeError:
            print(f"크기 {size}: 한계 도달")
            break
    
    print(f"최대 행렬 곱셈 크기: {max_matmul_size}x{max_matmul_size}")
    
    # 6. 메모리 처리량 한계 테스트
    print("\n6. 메모리 대역폭 한계")
    cleanup()
    
    sizes = [2048, 4096, 6144, 8192]
    for size in sizes:
        cleanup()
        try:
            # 메모리 복사 집약적 작업
            data = torch.randn(size, size, device=device)
            
            start_time = time.time()
            for _ in range(100):
                copied = data.clone()
                del copied
            torch.cuda.synchronize()
            
            copy_time = time.time() - start_time
            data_size_gb = (size * size * 4 * 100) / 1024**3
            bandwidth = data_size_gb / copy_time
            
            memory_used = get_memory_usage()
            print(f"크기 {size}: 대역폭 {bandwidth:.1f} GB/s, 메모리 {memory_used:.1f}MB")
            
            del data
            
        except RuntimeError:
            print(f"크기 {size}: 메모리 부족")
            break
    
    # 7. 최종 한계 요약
    print("\n" + "=" * 60)
    print("GTX 1060 3GB 절대 한계 요약")
    print("=" * 60)
    print(f"최대 단일 텐서 크기: {max_size}x{max_size}")
    print(f"최대 추론 배치 (224x224): {max_batch}")
    print(f"최대 훈련 배치 (224x224): {max_train_batch}")
    print(f"최대 행렬 곱셈: {max_matmul_size}x{max_matmul_size}")
    print("\n권장 안전 사용량:")
    print(f"- 추론 배치: {max_batch//2} 이하")
    print(f"- 훈련 배치: {max_train_batch//2} 이하")
    print(f"- 행렬 크기: {max_matmul_size//2} 이하")
    print("- 메모리 사용률: 80% 이하 유지")

if __name__ == "__main__":
    limit_performance_test()

GTX 1060 3GB 한계 성능 테스트
총 GPU 메모리: 3.00 GB

1. 최대 단일 텐서 크기 한계
크기 1000x1000: 11.9MB 성공
크기 1200x1200: 13.6MB 성공
크기 1400x1400: 15.6MB 성공
크기 1600x1600: 17.9MB 성공
크기 1800x1800: 20.5MB 성공
크기 2000x2000: 24.1MB 성공
크기 2200x2200: 26.6MB 성공
크기 2400x2400: 30.1MB 성공
크기 2600x2600: 34.1MB 성공
크기 2800x2800: 38.1MB 성공
크기 3000x3000: 42.5MB 성공
크기 3200x3200: 48.1MB 성공
크기 3400x3400: 52.2MB 성공
크기 3600x3600: 58.1MB 성공
크기 3800x3800: 64.1MB 성공
크기 4000x4000: 70.1MB 성공
크기 4200x4200: 76.1MB 성공
크기 4400x4400: 82.1MB 성공
크기 4600x4600: 88.8MB 성공
크기 4800x4800: 96.1MB 성공
크기 5000x5000: 104.1MB 성공
크기 5200x5200: 112.1MB 성공
크기 5400x5400: 120.1MB 성공
크기 5600x5600: 128.1MB 성공
크기 5800x5800: 136.5MB 성공
크기 6000x6000: 146.1MB 성공
크기 6200x6200: 154.8MB 성공
크기 6400x6400: 164.4MB 성공
크기 6600x6600: 174.3MB 성공
크기 6800x6800: 184.5MB 성공
크기 7000x7000: 195.0MB 성공
크기 7200x7200: 206.1MB 성공
크기 7400x7400: 217.0MB 성공
크기 7600x7600: 228.5MB 성공
크기 7800x7800: 240.2MB 성공
크기 8000x8000: 252.3MB 성공
크기 8200x8200: 264.6MB 성공
크기 8400x8400: 278.1MB 성공
크기 8600x8

In [3]:
import torch
print(torch.cuda.is_available())  # True가 출력되면 GPU 사용 가능
print(torch.cuda.get_device_name(0))  # GPU 이름

True
NVIDIA GeForce GTX 1060 3GB
